## Import Essential Library

In [1]:
#Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

#dataset import
dataset = pd.read_csv('data.csv') #You need to change #directory accordingly
dataset.head(10) #Return 10 rows of data

Marital status  Application mode  Application order  Course  \
0               1                17                  5     171   
1               1                15                  1    9254   
2               1                 1                  5    9070   
3               1                17                  2    9773   
4               2                39                  1    8014   
5               2                39                  1    9991   
6               1                 1                  1    9500   
7               1                18                  4    9254   
8               1                 1                  3    9238   
9               1                 1                  1    9238   

   Daytime/evening attendance  Previous qualification  \
0                           1                       1   
1                           1                       1   
2                           1                       1   
3                           1                       1   
4                           0                       1   
5                           0                      19   
6                           1                       1   
7                           1                       1   
8                           1                       1   
9                           1                       1   

   Previous qualification (grade)  Nacionality  Mother's qualification  \
0                           122.0            1                      19   
1                           160.0            1                       1   
2                           122.0            1                      37   
3                           122.0            1                      38   
4                           100.0            1                      37   
5                           133.1            1                      37   
6                           142.0            1                      19   
7                           119.0            1                      37   
8                           137.0           62                       1   
9                           138.0            1                       1   

   Father's qualification  ...  Curricular units 2nd sem (credited)  \
0                      12  ...                                    0   
1                       3  ...                                    0   
2                      37  ...                                    0   
3                      37  ...                                    0   
4                      38  ...                                    0   
5                      37  ...                                    0   
6                      38  ...                                    0   
7                      37  ...                                    0   
8                       1  ...                                    0   
9                      19  ...                                    0   

   Curricular units 2nd sem (enrolled)  \
0                                    0   
1                                    6   
2                                    6   
3                                    6   
4                                    6   
5                                    5   
6                                    8   
7                                    5   
8                                    6   
9                                    6   

   Curricular units 2nd sem (evaluations)  \
0                                       0   
1                                       6   
2                                       0   
3                                      10   
4                                       6   
5                                      17   
6                                       8   
7                                       5   
8                                       7   
9                                      14   

   Curricular units 2nd sem (approved)  Curricular units 2nd sem (grade)  \
0               

## Data Processing

In [2]:
dataset['Target'].value_counts()

Graduate    2209
Dropout     1421
Enrolled     794
Name: Target, dtype: int64

In [3]:
#Changing pandas dataframe to numpy array
X = dataset.iloc[:,:36].values
y = dataset.iloc[:,36:37].values
y[10:40]
print(X.shape)
print(y.shape)

(4424, 36)
(4424, 1)


In [4]:
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.combine import SMOTEENN
import math

bordersmote = SMOTEENN()
X, y = SMOTEENN().fit_resample(X, y)

In [5]:
unique = np.unique(y, return_counts = True)
unique
# (array(['Dropout', 'Enrolled', 'Graduate'], dtype=object),
# array([1020, 1416,  739], dtype=int64))

(array(['Dropout', 'Enrolled', 'Graduate'], dtype=object),
 array([1010, 1384,  730], dtype=int64))

## One - Hot Encoder

In [6]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# dummy_y[10:40]

## Normalize data

In [7]:
#Normalizing the data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

sc = StandardScaler()
X = sc.fit_transform(X)

poly_features = PolynomialFeatures(degree = 2, include_bias = False)
X = poly_features.fit_transform(X)

## Train Test Split

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size = 0.1)

## Build model

In [9]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
# Neural network
model = Sequential()
model.add(Dense(units = 36, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(units = 3, activation = 'softmax'))

In [10]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

## Training

In [11]:
history = model.fit(X_train, y_train, epochs = 300, validation_data = (X_test, y_test))

Epoch 1/300
88/88 [==============================] - 7s 24ms/step - loss: 0.8740 - accuracy: 0.6379 - val_loss: 0.6061 - val_accuracy: 0.7284
Epoch 2/300
88/88 [==============================] - 2s 18ms/step - loss: 0.5236 - accuracy: 0.7983 - val_loss: 0.5031 - val_accuracy: 0.8083
Epoch 3/300
88/88 [==============================] - 2s 18ms/step - loss: 0.3704 - accuracy: 0.8591 - val_loss: 0.4204 - val_accuracy: 0.8307
Epoch 4/300
88/88 [==============================] - 2s 19ms/step - loss: 0.2944 - accuracy: 0.8876 - val_loss: 0.4097 - val_accuracy: 0.8626
Epoch 5/300
88/88 [==============================] - 2s 17ms/step - loss: 0.2631 - accuracy: 0.9093 - val_loss: 0.3732 - val_accuracy: 0.8594
Epoch 6/300
88/88 [==============================] - 1s 17ms/step - loss: 0.2197 - accuracy: 0.9260 - val_loss: 0.4029 - val_accuracy: 0.8818
Epoch 7/300
88/88 [==============================] - 2s 18ms/step - loss: 0.1917 - accuracy: 0.9306 - val_loss: 0.4086 - val_accuracy: 0.8754
Epoch 

In [12]:
max(history.history['val_accuracy'])

0.9520766735076904

## Load model

In [13]:
from tensorflow.keras.models import load_model

In [14]:
model.save('predict_model.h5')

In [15]:
later_model = load_model('predict_model.h5')

In [16]:
losses = pd.DataFrame(model.history.history)

## Evaluate model

In [17]:
losses.plot()
plt.savefig('losses.png')

In [18]:
import random
random.seed(87)
random_ind = random.randint(0, len(dataset))

new_student = dataset.drop('Target', axis = 1).iloc[random_ind]
new_student

Marital status                                       1.000000
Application mode                                    17.000000
Application order                                    1.000000
Course                                            9853.000000
Daytime/evening attendance                           1.000000
Previous qualification                               1.000000
Previous qualification (grade)                     125.000000
Nacionality                                          1.000000
Mother's qualification                              19.000000
Father's qualification                              19.000000
Mother's occupation                                  4.000000
Father's occupation                                  7.000000
Admission grade                                    128.500000
Displaced                                            1.000000
Educational special needs                            0.000000
Debtor                                               0.000000
Tuition 

In [19]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [20]:
predictions = (model.predict(X_test) > 0.5).astype("int32")

10/10 [==============================] - 0s 6ms/step


In [21]:
print('Accuracy Score:', accuracy_score(y_test, predictions))
print('\n')
print(classification_report(y_test, predictions))

Accuracy Score: 0.9329073482428115


              precision    recall  f1-score   support

           0       0.97      0.90      0.93       101
           1       0.93      0.95      0.94       135
           2       0.92      0.95      0.94        77

   micro avg       0.94      0.93      0.94       313
   macro avg       0.94      0.93      0.94       313
weighted avg       0.94      0.93      0.94       313
 samples avg       0.93      0.93      0.93       313



c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
